##  Algorithm 2 in anechoic conditions
#### White noise localization

In [ ]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt 
import wave 
from scipy.io import wavfile
import soundfile
import time 
from scipy import signal
import logging
import datetime
from itertools import combinations
from Processing import calculate_angle_error

In [ ]:
def algorithm2(J=1,runs=10,SNR=20,step=1,hrtf=0):
    
    #input hrtf 0 for random 1 for lego 2 for kemar

    path_kemar = 'data/kemar_h_theta_1deg_time.npy' #  the path to the file
    path_lego = 'data/lego1_h_theta_time.npy'
    if hrtf==0:
        hrir = np.random.randn(T,D,M)
    if hrtf==1:
        hrir  = np.load(path_lego)
    if hrtf==2:
        hrir  = np.load(path_kemar) #head related transfer function
        
    T,D,M = hrir.shape
  

    fs = 16000 #samples/second
    T_ =0.5 #length of signal in time domain 
    N = int(T_*fs) #number of samples of the signal 

    mean = 0
    std = 1 

    obs_len = N + int(hrir.shape[0]) -1  #length of the convolution
 
    Df = int(hrir.shape[1]) #directions 

    avg_err = 0
    perfect = 0
    
    for rns in range(runs):
    
        St_all = np.zeros((N, J))
        for j in range(J):
            St_all[:, j] = np.random.normal(mean, std, size=N) #white noise

        
        theta = np.random.choice(range(Df), J, replace=False) #choose the directions randomly on the grid   
       
        yt = np.zeros((obs_len, )) #recorded time domain signal  voir deuxième paramètre 

        for j in range(J):
            yt += np.convolve(St_all[:, j], hrir[:, theta[j], 0]) #source signal convolved with corresponding directional response

        # Generate noise at required SNR    
        sig_norm = np.linalg.norm(yt)
        noise_t = np.random.randn(obs_len, ) #additive gaussian noise
        noise_norm = sig_norm/(10.**(SNR/20.))
        noise_t = noise_norm*noise_t/np.linalg.norm(noise_t)

        yt += noise_t #noisy signal
        f, t, y = signal.stft(yt, fs=fs, window='hann', nperseg=160)
        N_ = y.shape[1] #number of frames

        y_mean = np.mean(np.abs(y)**2, axis=1) #mean power frame
        y_mean = y_mean/np.linalg.norm(y_mean) #normalize the observation
    
        H = np.fft.rfft(hrir,axis=0)

        H_coarse = H[:,::step, :]
        D = H_coarse.shape[1]
        #search algorithm

        anglesf = np.arange(Df, dtype=np.int64)*360./Df # list of angles in degrees
        # Initialize variables
        best_ind = np.inf #index corresponding to best direction tuple
        smallest_norm = np.inf #smallest projection error
        best_dir = theta #best direction tuple
        
        # Search all combinations
        pairs2 = combinations(range(D), J) #all combinations in range(D) and length J=2
        for q2, d2 in enumerate(pairs2): 
    
            Bj = np.abs(H_coarse[:, d2, 0])**2 #vectors in current guess contains the two vectors 
            Pj = Bj.dot(np.linalg.pinv(Bj)) #projection matrix
            proj_error = np.linalg.norm((np.eye(int(y.shape[0])) - Pj).dot(y_mean)) #projection error

            if proj_error <= smallest_norm:
                smallest_norm = proj_error
                best_ind = q2
                best_dir = d2
            
        theta_hat = step*np.array(best_dir) #map coarse index to fine index
        min_err, best_perm = calculate_angle_error(theta, theta_hat, anglesf) #calculate error between chosen and true directions        
        avg_err += min_err 
        print('iteration ' + str(rns+1) +' error : '+ str(min_err) + '  ground truth : ' + str(theta)+ '  estimation : ' + str(best_perm) )    
        if min_err==0:
            perfect+=1
    
    avg_err = avg_err/runs
    print(str(J) +' sources')
    print('average error : ' + str(avg_err) +' over ' + str(runs) + ' runs')
    accuracy = (perfect/runs)*100
    print('accuracy : ' + str(accuracy) + ' %')
    
    
    return avg_err,accuracy


#### Call the function algorithm 2 
we fix the parameters : 

J : number of sources to detect   
runs : number of runs over which we calculate the average error   
SNR : signal to noise ration  
hrtf : to choose Lego, Kemar, or random data   

In [ ]:
runs1 = 1000 # for 1 source
runs2 = 100  # for 2 sources 
avg_err1, accuracy1   = backward_model_1(J=1, runs=runs1, SNR=10, step=1, hrtf=1)
avg_err2, accuracy2   = backward_model_1(J=1, runs=runs1, SNR=20, step=1, hrtf=1)
avg_err3, accuracy3   = backward_model_1(J=1, runs=runs1, SNR=30, step=1, hrtf=1)
avg_err4, accuracy4   = backward_model_1(J=2, runs=runs2, SNR=10, step=1, hrtf=1)
avg_err5, accuracy5   = backward_model_1(J=2, runs=runs2, SNR=20, step=1, hrtf=1)
avg_err6, accuracy6   = backward_model_1(J=2, runs=runs2, SNR=30, step=1, hrtf=1)
avg_err7, accuracy7   = backward_model_1(J=1, runs=runs1, SNR=10, step=1, hrtf=2)
avg_err8, accuracy8   = backward_model_1(J=1, runs=runs1, SNR=20, step=1, hrtf=2)
avg_err9, accuracy9   = backward_model_1(J=1, runs=runs1, SNR=30, step=1, hrtf=2)
avg_err10, accuracy10 = backward_model_1(J=2, runs=runs2, SNR=10, step=1, hrtf=2)
avg_err11, accuracy11 = backward_model_1(J=2, runs=runs2, SNR=20, step=1, hrtf=2)
avg_err12, accuracy12 = backward_model_1(J=2, runs=runs2, SNR=30, step=1, hrtf=2)